In [1]:
!pip install transformers
!pip install wandb -qU

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [2]:
import wandb
wandb.login(key="aec6fef7ba56ee445129472eb583718b8e529934")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random
import pickle
import os
import copy
import sys
from tqdm import tqdm
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, precision_score, recall_score, roc_curve, auc

In [4]:
def deflog(text):
    print(f"[DEF-LOG] {text}")

In [69]:
config['lr']

5e-05

In [5]:
config = {
  "lr": 5e-5,
  "weight_decay": 1e-8,
  "epoch_num": 100,
  "batch_size": 16
}
tags = ["new_data", "10percent", "BERT-Pretrained"]
run = wandb.init(entity='sentimovie', project="GGGGGGGOOOOVVVVVV", config=config, tags=tags)

wandb: Currently logged in as: sentimovie. Use `wandb login --relogin` to force relogin


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/mlops/mlops_course/phase2/reviews
#!cp -r "/content/drive/MyDrive/mlops/mlops_part_2_progect/reviews" "/content"

/content/drive/.shortcut-targets-by-id/1pV4N9G9PWwlYVi6Fer0Tq0HFxZXbrdmI/reviews


In [8]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

In [9]:
from transformers import AutoModel, BertTokenizerFast
#load model and tokenizer
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
tokenizer = BertTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
with open('train.pkl', 'rb') as f:
    train_set = pickle.load(f)
    
with open('test.pkl', 'rb') as f:
    test_set = pickle.load(f)

with open('val.pkl', 'rb') as f:
    val_set = pickle.load(f)

df_train = pd.DataFrame.from_dict(train_set)
df_test =  pd.DataFrame.from_dict(test_set)
df_val =  pd.DataFrame.from_dict(val_set)
df_train = df_train.dropna()
df_test = df_test.dropna()
df_val = df_val.dropna()

data = {'train': {}, "test": {}, 'val': {}}
data['train']['x'] = list(df_train['x'])[:4000]
data['train']['y'] = list(df_train['y'])[:4000]
data['test']['x'] = list(df_test['x'])[:1000]
data['test']['y'] = list(df_test['y'])[:1000]
data['val']['x'] = list(df_val['x'])[:1000]
data['val']['y'] = list(df_val['y'])[:1000]

del df_train
del df_test
del df_val
del train_set
del test_set
del val_set

In [11]:
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
set_seed(42)    

In [12]:
class BERT_architecture(nn.Module):
    def __init__(self, bert):
        super(BERT_architecture, self).__init__()
        self.bert = bert
        for param in bert.parameters():
            param.requires_grad = False

        # # dropout layer
        # self.dropout = nn.Dropout(0.2)
        # # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,50)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(50,2)

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, token_ids):
        _, cls_hs = self.bert(token_ids, return_dict=False)

        x = self.fc1(cls_hs)

        x = self.relu(x)

        #x = self.dropout(x)

        # output layer
        x = self.fc2(x)
      
        # apply softmax activation
        x = self.softmax(x)

        return x

In [13]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False


class MyDataset(Dataset):
    def __init__(self, X_data, labels):
        self.x_data = X_data
        self.labels = np.unique(labels, return_inverse=True)[1]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        max_length = 170
        label_onehot = torch.tensor(self.labels[idx])
        embedding = tokenizer(self.x_data[idx], return_tensors='pt', 
                              padding="max_length", max_length=max_length, truncation=True)['input_ids']
        embedding = embedding.squeeze()
        return embedding, label_onehot


def dataset_loader(data, config):
    dataset = {'train': MyDataset(data['train']['x'], data['train']['y']),
              'val': MyDataset(data['val']['x'], data['val']['y']),
              'test': MyDataset(data['test']['x'], data['test']['y'])}
    deflog("Datasets created")

    dataloaders = {x: DataLoader(dataset[x], batch_size=config['batch_size'], 
                                 num_workers=1) for x in ['train', 'val', 'test']}
    deflog("Dataloaders created")
    dataset_sizes = {x: len(dataset[x]) for x in ['train', 'val', 'test']}
    deflog("Dataset_sizes created")
    return dataloaders, dataset, dataset_sizes

In [14]:
def train_model(model, criterion, optimizer,  dataloaders, 
                dataset_sizes, device, config, earlyStopper, num_epochs=25):
    since = time.time()
    history = {'train_loss': [], 'val_loss': [], 
                    'train_acc': [], 'val_acc': []}
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            # if phase == 'train':
            #     scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(float(epoch_acc.cpu().numpy()))
            if phase == 'val':
              deflog(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            #######################################
            ###           wandb logging         ###
            #######################################
            ##        Log the training flow       ##
            #######################################
            tmp = {f'{phase}_loss': epoch_loss, 
                    f'{phase}_acc': epoch_acc,
                    'lr': config['lr'],
                    'epoch': epoch}
            wandb.log(tmp)
            #######################################
            #######################################

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        if earlyStopper.early_stop(history['val_loss'][-1]):
            deflog("Early Stopping")
            break

    time_elapsed = time.time() - since
    deflog(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    deflog(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

In [15]:
# dataset and dataloader
dataloaders, dataset, dataset_sizes = dataset_loader(data, config)

# model
model = BERT_architecture(bert)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
deflog(f"ML model defined (device: {device})")

earlyStopper = EarlyStopper(patience=10, min_delta=1e-2)
deflog("EarlyStopper defined")

class_weights = compute_class_weight('balanced', 
                                    classes=np.unique(data['train']['y']), 
                                    y=np.array(data['train']['y']))
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
deflog("Define loss class_weights")

optimizer = optim.AdamW(model.parameters(), lr=wandb.config['lr'],
                        weight_decay=wandb.config['weight_decay'])
deflog("Optimizer (Adam) defined")

# cosin_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, 
#                                                     T_max=config['epoch_num'], 
#                                                     eta_min=5e-6)
# deflog("Scheduler (cosin_lr_scheduler) defined")

model, history = train_model(model, criterion, optimizer,  
                              dataloaders, dataset_sizes, device, config,
                              earlyStopper, config['epoch_num'])

[DEF-LOG] Datasets created
[DEF-LOG] Dataloaders created
[DEF-LOG] Dataset_sizes created
[DEF-LOG] ML model defined (device: cuda:0)
[DEF-LOG] EarlyStopper defined
[DEF-LOG] Define loss class_weights
[DEF-LOG] Optimizer (Adam) defined


100%|██████████| 63/63 [00:09<00:00,  6.36it/s]


[DEF-LOG] val Loss: 0.6926 Acc: 0.6440


100%|██████████| 63/63 [00:10<00:00,  5.97it/s]


[DEF-LOG] val Loss: 0.6932 Acc: 0.6440


...

[DEF-LOG] val Loss: 0.6937 Acc: 0.4120


100%|██████████| 63/63 [00:11<00:00,  5.30it/s]


[DEF-LOG] val Loss: 0.6911 Acc: 0.6430


 65%|██████▌   | 41/63 [00:07<00:04,  5.15it/s]


KeyboardInterrupt: ignored

In [ ]:
# #dataloaders, dataset, dataset_sizes = dataset_loader(data, config)
# t, l = next(iter(dataloaders['test']))
# # input_s = input['input_ids'].to(device)
# t0 = t.to(device)
# model(t0)
# t.shape, l.shape

[DEF-LOG] Datasets created
[DEF-LOG] Dataloaders created
[DEF-LOG] Dataset_sizes created


(torch.Size([128, 170]), torch.Size([128]))

In [ ]:
# # get predictions for test data
# with torch.no_grad():
#   preds = model(test_seq.to(device), test_mask.to(device))
#   preds = preds.detach().cpu().numpy()

In [ ]:

# dataloaders["test"]

In [ ]:
# model.eval()  
# for inputs, labels in tqdm(dataloaders["test"]):
#       inputs = inputs.to(device)
#       labels = labels.to(device)
#       outputs = model(inputs)


  1%|          | 1/186 [00:00<01:14,  2.48it/s]


IndexError: ignored

In [ ]:

# from sklearn.metrics import classification_report

In [ ]:
# pred = np.argmax(preds, axis = 1)
# print(classification_report(test_y, pred))